### Imports

In [50]:
from statsforecast import StatsForecast 
from statsforecast.models import AutoETS, AutoARIMA

In [51]:
import pandas as pd 

### Cross Validation

In [52]:
Y_df = pd.read_parquet('data/m5-dataset.parquet.gzip')

In [53]:
Y_df.head()

,unique_id,ds,y
0,FOODS_1_001_CA_1,2011-01-29,3.0
1,FOODS_1_001_CA_1,2011-01-30,0.0
2,FOODS_1_001_CA_1,2011-01-31,0.0
3,FOODS_1_001_CA_1,2011-02-01,1.0
4,FOODS_1_001_CA_1,2011-02-02,4.0


In [54]:
StatsForecast.plot(Y_df)

In [55]:
df = Y_df[Y_df['unique_id'] == "HOUSEHOLD_1_429_TX_3"] # select time series

In [56]:
models = [AutoETS(season_length=14)]

sf = StatsForecast(
    df = df, 
    models = models, 
    freq = 'D', 
    n_jobs = -1
)

In [57]:
crossvalidation_df = sf.cross_validation(
    df = df,
    h = 14,
    step_size = 14,
    n_windows = 3
  )

In [58]:
crossvalidation_df.head()

,ds,cutoff,y,AutoETS
unique_id,,,,
HOUSEHOLD_1_429_TX_3,2016-04-11,2016-04-10,4.0,8.143165
HOUSEHOLD_1_429_TX_3,2016-04-12,2016-04-10,10.0,6.214600
HOUSEHOLD_1_429_TX_3,2016-04-13,2016-04-10,8.0,6.092824
HOUSEHOLD_1_429_TX_3,2016-04-14,2016-04-10,6.0,6.035365
HOUSEHOLD_1_429_TX_3,2016-04-15,2016-04-10,5.0,6.807395


In [59]:
crossvalidation_df.rename(columns = {'y' : 'actual'}, inplace = True) # rename actual values 

cutoff = crossvalidation_df['cutoff'].unique()

for k in range(len(cutoff)): 
    cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
    StatsForecast.plot(df, cv.loc[:, cv.columns != 'cutoff'])